In [ ]:
from datasets import load_dataset
dataset = load_dataset("text", data_files={"train": "ner_auto_labels.conll", "validation": "ner_auto_labels.conll"})

In [ ]:
import os 
import sys 
sys.path.append(os.path.abspath(os.path.join('..')))
import scripts.fine_tune as ft

data = ft.parse_conll("ner_auto_labels.conll")

In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_list(data)
split_dataset = dataset.train_test_split(test_size=0.2)
datasets = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

In [ ]:
label_list = sorted(set(tag for row in datasets["train"]['ner_tags'] for tag in row))
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

In [ ]:
from datasets import Dataset, DatasetDict

# Split into train and validation (e.g., 90/10)
train_size = int(0.9 * len(data)) # Corrected variable name
train_data = data[:train_size]
val_data = data[train_size:]

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data)
})

In [ ]:
unique_tags = set(tag for sentence in data for tag in sentence['ner_tags'])
label_list = sorted(unique_tags)
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
from transformers import AutoTokenizer

model_name = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label[word_idx].startswith("I-") else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

In [ ]:
%pip install torch

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

In [ ]:
%pip install "accelerate>=0.26.0" --upgrade

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (usually -100)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import Trainer, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator, # Added data collator for padding
    compute_metrics=compute_metrics,

)

train model

In [ ]:
trainer.train()


Evaluate model

In [ ]:
trainer.evaluate()